# Reusable Codes in Python

<b style="font-size:20px"> <a href="#Create py file and store variable">Create py file and store variable</a><br>
<b style="font-size:20px"> <a href="#Import IPython Notebooks as Module">Import IPython Notebooks as Module</a>

<b style="font-size:15px"><a id="Create py file and store variable">Create py file and store variable</a>

In [1]:
"""
Create py file to store variable
To include save_var.py in other code use "from save_var import *"
"""

f = open( 'save_var.py', 'w' ) # Put this in the beginning of your program. It will create a new file
f.close()

# call save_var and pass the variable you want to save
def save_var(var):
    var_name = [name for name in globals() if globals()[name] is var][0] # fetch variable name
        
    f = open( 'save_var.py', 'a' ) # append to the file created
    f.write('#' + repr(type(var)) + '\n' )
    f.write('#Len:' + repr(len(var)) + '\n' )
    f.write( var_name + ' = ' + repr(var) + '\n\n' )
    
    f.close()

<b style="font-size:15px"><a id="Import IPython Notebooks as Module">Import IPython Notebooks as Module</a>

In [1]:
"""
Import IPython Notebooks as Module.
http://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Importing%20Notebooks.html
http://nbviewer.jupyter.org/gist/minrk/6011986
"""

import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

# Module to import
#from bigram_hmm import BigramHMM